In [2]:
## Exercise - Decision Tree

In [3]:
import numpy as np
import pandas as pd
import math
eps = np.finfo(float).eps
from numpy import log2 as log
from sklearn.tree import DecisionTreeClassifier

In [4]:
import collections

In [5]:
## Input the Mushrooms Example Data

mushrooms = [{"Color": "red", "Size": "small", "Points": "yes", "Eatability": "toxic"},
{"Color": "brown", "Size": "small", "Points": "no", "Eatability": "edible"},
{"Color": "brown", "Size": "large", "Points": "yes", "Eatability": "edible"},
{"Color": "green", "Size": "small", "Points": "no", "Eatability": "edible"},
{"Color": "red", "Size": "large", "Points": "no", "Eatability": "edible"}]


In [6]:
## Converting the dataset to dataframe
mushrooms = pd.DataFrame(mushrooms,columns=['Color','Size','Points','Eatability'])

In [7]:
mushrooms

,Color,Size,Points,Eatability
0,red,small,yes,toxic
1,brown,small,no,edible
2,brown,large,yes,edible
3,green,small,no,edible
4,red,large,no,edible


In [8]:
##converting the attributes into numeric format with the use of LabelEncoder

In [9]:
from sklearn.preprocessing import LabelEncoder 

In [10]:
LabelEncoder_mushrooms = LabelEncoder()

In [11]:
mushroom = mushrooms.apply(LabelEncoder().fit_transform)

In [12]:
mushroom

,Color,Size,Points,Eatability
0,2,1,1,1
1,0,1,0,0
2,0,0,1,0
3,1,1,0,0
4,2,0,0,0


In [13]:
# Separating each of the attributes for ease of compting the entropy
target = mushrooms['Eatability']
attribute_Color = mushrooms['Color']
attribute_Size = mushrooms['Size']
attribute_Points = mushrooms['Points']

In [14]:
## Calculating the entropy of the entire dataset
def total_entropy(target):
    entropy = 0
    target_A = np.unique(target)
    for a_ in target_A:
        target_B = float(np.count_nonzero (target == a_)) / target.size
        entropy += target_B * np.log2(1/target_B)
    return entropy

In [15]:
total_entropy(target)

0.7219280948873623

In [16]:
##Calculating the Conditional Entropy of the first attribute (Color)

In [17]:
attribute = 'Color'

In [18]:
target= mushrooms.Eatability.unique()  

In [19]:
attribute_1 = mushrooms[attribute].unique()

In [20]:
entropy_attribute_Color = 0
for attributes in attribute_1:
    entropy_data = 0
    for target_attributes in target:
        num = len(mushrooms[attribute][mushrooms[attribute]==attributes][mushrooms.Eatability ==target_attributes]) 
        den = len(mushrooms[attribute][mushrooms[attribute]==attributes]) 
        p1 = num/(den+eps)  
        entropy_data += -p1*log(p1+eps)
    p2 = den/len(mushrooms)
    entropy_attribute_Color += abs(-p2*entropy_data)

In [21]:
entropy_attribute_Color

0.39999999999999986

In [22]:
##Calculating the Conditional Entropy of the second attribute (Size)
attribute_S = 'Size'

In [23]:
attribute_S1 = mushrooms[attribute_S].unique()

In [24]:
entropy_attribute_Size = 0
for attributes_S in attribute_S1:
    entropy_data = 0
    for target_attributes_S in target:
        num = len(mushrooms[attribute_S][mushrooms[attribute_S]==attributes_S][mushrooms.Eatability ==target_attributes_S]) 
        den = len(mushrooms[attribute_S][mushrooms[attribute_S]==attributes_S]) 
        p1 = num/(den+eps)  
        entropy_data += -p1*log(p1+eps)
    p2 = den/len(mushrooms)
    entropy_attribute_Size += abs(-p2*entropy_data)

In [25]:
entropy_attribute_Size

0.5509775004326934

In [26]:
##Calculating the Conditional Entropy of the second attribute (Size)
attribute_P = 'Points'

In [27]:
attribute_P1 = mushrooms[attribute_P].unique()

In [28]:
entropy_attribute_Points = 0
for attributes_P in attribute_P1:
    entropy_data = 0
    for target_attributes_P in target:
        num = len(mushrooms[attribute_P][mushrooms[attribute_P]==attributes_P][mushrooms.Eatability ==target_attributes_P]) 
        den = len(mushrooms[attribute_P][mushrooms[attribute_P]==attributes_P]) 
        p1 = num/(den+eps)  
        entropy_data += -p1*log(p1+eps)
    p2 = den/len(mushrooms)
    entropy_attribute_Points += abs(-p2*entropy_data)

In [29]:
entropy_attribute_Points

0.3999999999999999

In [30]:
## Creating a loop that is capable of returning the conditional entropy of each attributes 

def conditional_entropy(mushrooms, attribute,target):
    Target_Label = mushrooms.keys()[-1]   
    target = mushrooms[Target_Label].unique()  
    Attributes_1 = mushrooms[attribute].unique()
    ent_2 = 0
    for Attributes in Attributes_1:
        ent = 0
        for targetAttrib in target:
            num = len(mushrooms[attribute][mushrooms[attribute]==Attributes][mushrooms[Target_Label] ==targetAttrib])
            den = len(mushrooms[attribute][mushrooms[attribute]==Attributes])
            p1 = num/(den+eps)
            ent += -p1*log(p1+eps)
        p2 = den/len(mushrooms)
        ent_2 += -p2*ent
    return abs(ent_2)

In [31]:
conditional_entropy(mushrooms, attribute,target)

0.39999999999999963

In [32]:
## Determining the attribute with the highest Infromation Gain(IG) 
def Highest_IG(mushrooms):
    ## Creating objects for the Entropy of the Attributes
    EntAttrb = []
    Information_Gain = []    
    for members in mushrooms.keys()[:-1]:
        Information_Gain.append(total_entropy(target)- conditional_entropy(mushrooms,members,target))
    return mushrooms.keys()[:-1][np.argmax(Information_Gain)]

In [33]:
Highest_IG(mushrooms)

'Points'

In [34]:
class ID3Node:
    def create_edge(mushrooms,attribute_value,Node):
        return mushrooms[mushrooms[attribute_value] == Node].reset_index(drop=True)    
    def classify(mushrooms,target):
        mushrooms.attribute = None  
        mushrooms.attribute_values = [] 
        mushrooms.target = target   
        mushrooms.classify = {}   
        mushrooms.label= false
        mushrooms.instances_labeled = []

In [35]:
## Building the decision tree
def lower_tree(mushrooms,attribute_value,Node):
    return mushrooms[mushrooms[attribute_value] == Node].reset_index(drop=True)

def id3(mushrooms,tree=None): 
    Target_Label = mushrooms.keys()[-1]   
    attribute_value = Highest_IG(mushrooms)    #Setting node to attribute with highest information gain (Highest_IG(mushrooms))
    Node_attribute = np.unique(mushrooms[attribute_value])
        
    if tree is None:                                   
        tree={}
        tree[attribute_value] = {} 
    for Node in Node_attribute:     # loop that stops when the subset became pure, (that is, leave node)
        subset = lower_tree(mushrooms,attribute_value,Node)
        Node_leave,counts = np.unique(subset['Eatability'],return_counts=True)                        
        #Checking purity of subset
        if len(counts)==1:
            tree[attribute_value][Node] = Node_leave[0]                                                    
        else:        
            tree[attribute_value][Node] = id3(subset) 
    return tree 

In [36]:
id3(mushrooms)         #Printing the decision tree output

{'Points': {'no': 'edible',
  'yes': {'Color': {'brown': 'edible', 'red': 'toxic'}}}}

In [37]:
tree = id3(mushrooms)            #Tree output

In [38]:
## To verify the prediction of the implementation, using the first row of the example ({'Color': 'red', 'Points': 'yes', 'Size': 'small'})
##'toxic')
## Set the first row to be row_1

In [39]:
row_1 = mushrooms.iloc[0]

In [40]:
#The function predict based on the input variable

def classify(row_1, tree):
     
    for attribute_value in tree.keys():        
        
        Node = row_1[attribute_value]           ## Setting the node/child node to row_1 to determine its leave(target) value 
        tree = tree[attribute_value][Node]
        label = 0
                    
        if type(tree) is dict:
            label = classify(row_1, tree)
        else:
            label = tree
            break;                            
        
    return label

In [41]:
classify(row_1, tree)

'toxic'

In [42]:
## Similarly, checking for the second row_2 of the example {'Color': 'brown', 'Points': 'yes', 'Size': 'large'})
## by just replacing row_1 with row_2

In [43]:
row_2 = mushrooms.iloc[1]

In [44]:
def classify(row_2, tree):
     
    for attribute_value in tree.keys():        
        
        Node = row_2[attribute_value]           ## Setting the node/child node to row_1 to determine its leave(target) value 
        tree = tree[attribute_value][Node]
        label = 0
                    
        if type(tree) is dict:
            label = classify(row_2, tree)
        else:
            label = tree
            break;                            
        
    return label

In [45]:
classify(row_2, tree)

'edible'

In [46]:
## Comment on misclassification rate
## Since all the label data for the expected output are similar, the misclassification rate is ZERO (0%)

In [47]:
## Implementing the functions with the Adult Dataset

In [48]:
## Importing the Adult dataset, and using Pandas 

In [49]:
#load path to the adult dataset
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',sep= ',',names =['Age','Workclass','Fnlwgt','Education','Education-num', 'Marital-status','Occupation', 'Relationship','Race', 'Sex', 'Capital-gain', 'Capital-loss','Hours-per-week','Native-country','Income'])

In [50]:
data

,Age,Workclass,Fnlwgt,Education,Education-num,Marital-status,Occupation,Relationship,Race,Sex,Capital-gain,Capital-loss,Hours-per-week,Native-country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [51]:
data.shape

(32561, 15)

In [52]:
data.head()

,Age,Workclass,Fnlwgt,Education,Education-num,Marital-status,Occupation,Relationship,Race,Sex,Capital-gain,Capital-loss,Hours-per-week,Native-country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [53]:
data.tail()

,Age,Workclass,Fnlwgt,Education,Education-num,Marital-status,Occupation,Relationship,Race,Sex,Capital-gain,Capital-loss,Hours-per-week,Native-country,Income
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
32560,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K


In [54]:
data.describe()

,Age,Fnlwgt,Education-num,Capital-gain,Capital-loss,Hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [55]:
## removing the columns with "continuos" attributes 

In [56]:
data = data.drop(['Age','Fnlwgt','Education-num','Capital-gain','Capital-loss','Hours-per-week'], axis=1)

In [57]:
data      ## Remaining dataset

,Workclass,Education,Marital-status,Occupation,Relationship,Race,Sex,Native-country,Income
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,<=50K
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,<=50K
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...
32556,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,United-States,<=50K
32557,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States,>50K
32558,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,United-States,<=50K
32559,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,United-States,<=50K


In [58]:
## Spliting the dataset into training set (the first 20,000 rows) and test set (the remaining 12,561)

In [59]:
training_data = data[:20000]       ## training dataSset

In [60]:
test_data = data[20001:]           ## test dataset

In [61]:
## Obtaining the total entropy of the dataset

In [62]:
target = training_data['Income']

In [63]:
def total_entropy(target):
    entropy = 0
    target_A = np.unique(target)
    for a_ in target_A:
        target_B = float(np.count_nonzero (target == a_)) / target.size
        entropy += target_B * np.log2(1/target_B)
    return entropy

In [64]:
total_entropy(target)

0.7917824316527677

In [65]:
##Calculating the Conditional Entropy of the entire attributes

In [66]:
attribute = 'Workclass'

In [67]:
def conditional_entropy(training_data, attribute,target):
    Target_Label = training_data.keys()[-1]      ## removing the "target" column  
    target = training_data[Target_Label].unique()  
    Attributes_1 = training_data[attribute].unique()
    ent_2 = 0
    for Attributes in Attributes_1:
        ent = 0
        for targetAttrib in target:
            num = len(training_data[attribute][training_data[attribute]==Attributes][training_data[Target_Label] ==targetAttrib])
            den = len(training_data[attribute][training_data[attribute]==Attributes])
            p1 = num/(den+eps)
            ent += -p1*log(p1+eps)
        p2 = den/len(training_data)
        ent_2 += -p2*ent
    return abs(ent_2)

In [68]:
conditional_entropy(training_data, attribute,target)

0.7706284267318722

In [69]:
## Determining the attribute with the highest Infromation Gain(IG) 
def Highest_IG(training_data):
    ## Creating objects for the Entropy of the Attributes
    EntAttrb = []
    Information_Gain = []    
    for members in training_data.keys()[:-1]:
        Information_Gain.append(total_entropy(target)- conditional_entropy(training_data, members,target))
    return training_data.keys()[:-1][np.argmax(Information_Gain)]

In [70]:
Highest_IG(training_data)

'Relationship'

In [71]:
## from the above outcome, "Relationship is the root node"

In [72]:
class ID3Node:
    def create_edge(training_data,attribute_value,Node):
        return training_data[training_data[attribute_value] == Node].reset_index(drop=True)    
    def classify(training_data,target):
        training_data.attribute = None  
        training_data.attribute_values = [] 
        training_data.target = target   
        training_data.classify = {}   
        training_data.label= false
        training_data.instances_labeled = []

In [73]:
## Building the decision tree
def lower_tree(training_data,attribute_value,Node):
    return training_data[training_data[attribute_value] == Node].reset_index(drop=True)

def id3(training_data,tree=None): 
    Target_Label = training_data.keys()[-1]   
    attribute_value = Highest_IG(training_data)    #Setting node to the attribute with highest information gain (Highest_IG(mushrooms))
    Node_attribute = np.unique(training_data[attribute_value])
        
    if tree is None:                                   
        tree={}
        tree[attribute_value] = {} 
    for Node in Node_attribute:     # loop that stops when the subset became pure, (that is, leave node)
        subset = lower_tree(training_data,attribute_value,Node)
        Node_leave,counts = np.unique(subset['Income'],return_counts=True)                        
        #Checking purity of subset
        if len(counts)==1:
            tree[attribute_value][Node] = Node_leave[0]                                                    
        else:        
            tree[attribute_value][Node] = id3(subset) 
    return tree

In [74]:
## Verifying the prediction of the implementation
## using the first row (Set the first row to be row_1)

In [75]:
train_target = training_data.Income     ##Separating the attribute data from target (in both training_data and test_data)
test_target = test_data.Income
train_attributes = training_data.iloc[:,0:8]     ##The first eight column is the attribute
test_attributes = test_data.iloc[:,0:8]

In [76]:
Classify = DecisionTreeClassifier()

In [77]:
train_attributes = train_attributes.apply(LabelEncoder().fit_transform)
test_attributes = test_attributes.apply(LabelEncoder().fit_transform)
Classify.fit(train_attributes, train_target)

DecisionTreeClassifier()

In [78]:
input = np.array(test_attributes)        ##inputing the test_attributes

In [79]:
Target_predict = Classify.predict(input)

In [80]:
Target_predict

array([' >50K', ' <=50K', ' <=50K', ..., ' <=50K', ' <=50K', ' >50K'],
      dtype=object)